# 损失函数与反向传播、优化器
---
### 1. nn.L1loss
https://pytorch.org/docs/stable/generated/torch.nn.L1Loss.html#torch.nn.L1Loss


class torch.nn.L1Loss(size_average=None, reduce=None, reduction='mean')

Parameters:
+ size_average (bool, optional) – Deprecated (see reduction). By default, the losses are averaged over each loss element in the batch. Note that for some losses, there are multiple elements per sample. If the field size_average is set to False, the losses are instead summed for each minibatch. Ignored when reduce is False. Default: True

+ reduce (bool, optional) – Deprecated (see reduction). By default, the losses are averaged or summed over observations for each minibatch depending on size_average. When reduce is False, returns a loss per batch element instead and ignores size_average. Default: True

 + reduction (str, optional) – Specifies the reduction to apply to the output: 'none' | 'mean' | 'sum'. 'none': no reduction will be applied, 'mean': the sum of the output will be divided by the number of elements in the output, 'sum': the output will be summed. Note: size_average and reduce are in the process of being deprecated, and in the meantime, specifying either of those two args will override reduction. Default: 'mean'



In [18]:
import torch 
from torch.nn import L1Loss

inputs = torch.tensor([1, 2, 3], dtype=torch.float32)
targets = torch.tensor([1, 2, 5], dtype=torch.float32)

inputs  = torch.reshape(inputs, (1, 1, 1, 3))
targets = torch.reshape(targets, (1, 1, 1, 3))

# 初始化一个实例若不填入任何参数则默认的reduction是'mean'
loss = L1Loss()
result = loss(inputs, targets)

print(result)

tensor(0.6667)


☝实际上就是对应位置直接相减，求平均值。

In [19]:
loss = L1Loss(reduction='sum')
result = loss(inputs, targets)

print(result)

tensor(2.)


☝实际上就是对应位置直接相减，求总和。

### 2. nn.MSELoss
---
均方差，`mean square error`   
https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss

In [20]:
from torch import nn

loss_mse = nn.MSELoss()
result_mse = loss_mse(inputs, targets)

print(result_mse)

tensor(1.3333)


### 3. nn.CrossEntropyLoss
---
+ 处理分类问题

https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss

In [21]:
x = torch.tensor([0.1, 0.2, 0.3])
y = torch.tensor([1])

In [22]:
x = torch.reshape(x, (1, 3))

In [23]:
loss_cross = nn.CrossEntropyLoss()
result_cross = loss_cross(x, y)
print(result_cross)

tensor(1.1019)


### 4. .backward() 反向传播, 优化器
---
+ 计算推理结果和目标之间的差距
+ 给反向传播提供了依据   
SGD优化器：   
https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD

In [41]:
from torch.nn import Sequential, Conv2d, MaxPool2d, Flatten, Linear
from torch import nn
import torchvision
from torch.utils.data import DataLoader
import torch

class Tudui(nn.Module):
    
    def __init__(self):
        super(Tudui, self).__init__()
        self.model1 = Sequential(
        Conv2d(3, 32, 5, padding = 2),
        MaxPool2d(2),
        Conv2d(32, 32, 5, padding=2),
        MaxPool2d(2),
        Conv2d(32, 64, 5, padding=2),
        MaxPool2d(2),
        Flatten(),
        Linear(1024, 64),
        Linear(64, 10)
        )
        
    def forward(self, x):
        x = self.model1(x)
        return x

In [42]:
dataset = torchvision.datasets.CIFAR10("../dataset", train=False, transform=torchvision.transforms.ToTensor(), download=True)
dataloader = DataLoader(dataset, batch_size=1, drop_last=True)

Files already downloaded and verified


In [44]:
loss = nn.CrossEntropyLoss()
tudui = Tudui()
optim = torch.optim.SGD(tudui.parameters(), lr=0.01)

for epoch in range(20):
    running_loss = 0.0
    # 在原来的数据上进行一轮的学习
    for data in dataloader:
        imgs, targets = data
        outputs = tudui(imgs)     # 将imgs图像通过神经网络得到输出
        # print("Target is",targets)                       
        # print("Output is", outputs)
        result_loss = loss(outputs, targets)    # 计算出目标值和真实值之间的交叉熵损失
        # print("The CrossEntropyLoss is",result_loss)
        optim.zero_grad()              # 上一轮的梯度对本轮训练没有帮助，要清0
        result_loss.backward()    #  用当前的result反向传播修改参数
        optim.step()
        running_loss = running_loss + result_loss
        
    print(running_loss)

tensor(18804.1738, grad_fn=<AddBackward0>)
tensor(16166.6309, grad_fn=<AddBackward0>)
tensor(15508.4863, grad_fn=<AddBackward0>)
tensor(16208.1094, grad_fn=<AddBackward0>)
tensor(17930.9062, grad_fn=<AddBackward0>)
tensor(20512.9863, grad_fn=<AddBackward0>)
tensor(22462.5352, grad_fn=<AddBackward0>)
tensor(23612.9922, grad_fn=<AddBackward0>)
tensor(24438.5137, grad_fn=<AddBackward0>)
tensor(25003.1914, grad_fn=<AddBackward0>)
tensor(25845.9453, grad_fn=<AddBackward0>)
tensor(26196.1777, grad_fn=<AddBackward0>)
tensor(26821.2383, grad_fn=<AddBackward0>)
tensor(27672.5293, grad_fn=<AddBackward0>)
tensor(28235.2109, grad_fn=<AddBackward0>)
tensor(29414.8438, grad_fn=<AddBackward0>)
tensor(32945.1914, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
